## Imports

In [2]:
import h5py
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
from keras import Input, Model
import json

### Load Data

In [3]:
def load_data(path):
    with h5py.File(path, 'r') as f:
        x = f['image'][()]
        y = f['label'][()]
    return np.array(x), np.array(y)

# Set this to none if you will manually input the needed variables
dataJson = r"C:\Users\Michael\Desktop\Model Benchmark\data\CatOrDog\data.json"

# Image dimensions that the model was trained on
# Manually Set This If data.json does not exist, otherwise can leave as None
# input_dimensions = [96, 96, 1]
input_dimensions = None

# Class Names
# Manually Set This If data.json does not exist, otherwise can leave as None
# class_names = ["Cat", "Dog"]
class_names = None

# Number of classes
# Manually Set This If data.json does not exist, otherwise can leave as None
# num_classes = 2
num_classes = None

# Automatically read the input_dimensions, class_names and num_classes from a data.json if it exists
if dataJson is not None:
    with open(dataJson, "r") as f:
        data = json.load(f)

    input_dimensions = data['input_size']
    class_names = data['class_names']
    num_classes = data['num_classes']

    f.close()

# Load data
x_train, y_train = load_data(r'C:\Users\Michael\Desktop\Model Benchmark\data\CatOrDog\train.h5')
x_val, y_val = load_data(r'C:\Users\Michael\Desktop\Model Benchmark\data\CatOrDog\val.h5')

# Normalize data
x_train = x_train.reshape(-1, input_dimensions[0], input_dimensions[1], input_dimensions[2]) / 255.0
x_val = x_val.reshape(-1, input_dimensions[0], input_dimensions[1], input_dimensions[2]) / 255.0

y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)

Initialize Model

In [5]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


inp = Input(shape=(96, 96, 1))

k = 3
s = 1
# inpShape = x.shape
x = Conv2D(128, k, s, activation='relu', padding="valid")(inp)
# print("Valid Padding:")
# print(x.shape)
# print(compute_output_shape(inpShape, k, s))

x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

#test

# Block 1
k = 3
s = 1
# inpShape = x.shape
x = Conv2D(256, k, s, activation='relu', padding="valid")(x)
# print("Valid Padding:")
# print(x.shape)
# print(compute_output_shape(inpShape, k, s))

x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

# Block 1
k = 7
s = 1
# # inpShape = x.shape
x = Conv2D(512, k, s, activation='relu', padding="valid")(x)


x = Flatten()(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
out = Dense(2, activation='softmax')(x)

model = Model(inputs=inp, outputs=out, name="cat_dog_model_standard")
print(model.summary())

Model: "cat_dog_model_standard"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 96, 96, 128)       1280      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 48, 48, 128)      0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 48, 48, 256)       295168    
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 24, 24, 256)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 

## Train Model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('best_standard_model.h5', monitor='val_loss', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Fit model
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_val, y_val),
    epochs=50,
    callbacks=[checkpoint, early_stopping]
)

# Save the whole model at the end
# model.save('standardModel2.h5')